In [81]:
from selenium import webdriver
import selenium
import time
import random
import os
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from PIL import Image
from io import BytesIO
import requests

In [82]:
# 사용자 인터페이스 설정
print('+'*100)
print('지마켓 베스트셀러 크롤러')
print('-'*100)

cnt = int(input('1.지마켓 베스트셀러 중 몇 건을 수집하시겠습니까? : '))
f_dir = input('2.파일저장경로를 입력하세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+'지마켓_베스트셀러')
os.chdir(f_dir+s+'-'+'지마켓_베스트셀러')

f_name = s+'-'+'지마켓_베스트셀러'+'.txt'
fc_name = s+'-'+'지마켓_베스트셀러'+'.csv'
fx_name = s+'-'+'지마켓_베스트셀러'+'.xlsx'

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
지마켓 베스트셀러 크롤러
----------------------------------------------------------------------------------------------------
1.지마켓 베스트셀러 중 몇 건을 수집하시겠습니까? : 200
2.파일저장경로를 입력하세요 : 


In [83]:
# 드라이버 옵션 설정 및 변수에 담기 
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')

driver = webdriver.Chrome(path, options=options)

#1. 시작 타임스탬프 찍기 
s_time = time.time()
#2. 페이지 접속
driver.get('http://corners.gmarket.co.kr/Bestsellers')
time.sleep(5)


In [84]:
#3. 스크롤 페이지 끝까지 내리기 
for a in range(30) : 
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(5)

#4. 페이지 전체 파싱하기 
html0 = driver.page_source
soup0 = BeautifulSoup(html0, 'html.parser')
con_list0 = soup0.find_all('div', 'best-list')[1]
con_list = con_list0.find_all('li')

#5. for 문 돌면서 각 항목 수집하기 

no = 1
f = open(f_name, 'a', encoding='utf-8')
no_list1 = []
title1 = []
o_price1 = []
s_price1 = []
s_rate1 = []
img_list1 = []

for b in con_list : 
    print('-'*100)
    f.write('-'*100)
    
    #판매순위
    print('1.판매순위: %s' %no)
    f.write('1.판매순위: %s'%no+'\n')
    no_list1.append(no)
    
    #이미지 src 수집
    img_src = b.find('img')['src']
    img_list1.append(img_src)
    
    #제품소개 
    title = b.find('a', 'itemname').get_text().replace('\n', "").strip()
    title1.append(title)
    print('2.제품소개: %s' %title)
    f.write('2.제품소개: %s' %title+'\n')
    
    #원래가격
    try : 
        o_price = b.find('div', 'o-price').get_text().replace('\n', "").strip()
        o_price1.append(o_price)
        print('3.원래가격: %s' %o_price)
        f.write('3.원래가격: %s' %o_price+'\n')
    except : 
        print('3.원래가격: 제공된 정보 없음')
        f.write('3.원래가격: 제공된 정보 없음'+'\n')
        o_price1.append(' ')
    
    #판매가격
    s_price = b.find('div', 's-price').find('span').get_text().replace('\n', "").strip()
    s_price1.append(s_price)
    print('4.판매가격: %s' %s_price)
    f.write('4.판매가격: %s' %s_price+'\n')
    
    #할인율
    try : 
        span_list = b.find('div', 's-price').find_all('span')
        s_rate = span_list[2].get_text().replace('\n', "").strip()
        s_rate1.append(s_rate)
        print('5.할인율: %s' %s_rate)
        f.write('5.할인율: %s' %s_rate+'\n')
    except : 
        print('5.할인율: 제공된 정보 없음')
        f.write('5.할인율: 제공된 정보 없음'+'\n')
        s_rate1.append(' ')
        
    f.write('\n')
    print()
    f.write('-'*100+'\n')
    print('-'*100)
    
    no += 1 
    #6. 갯수만큼 수집했으면 break로 for문 종료 
    if no > cnt : 
        break
    elif no-1 == len(con_list) : 
        break
    
f.close()
driver.quit()


#7. 데이터프레임 만들기 
g = pd.DataFrame()
g['판매순위'] = no_list1
g['제품소개'] = title1
g['원래가격'] = o_price1
g['판매가격'] = s_price1
g['할인율'] = s_rate1

#8.데이터프레임 저장하기 
g.to_csv(fc_name, encoding='utf-8-sig')
g.to_excel(fx_name)

#9.엑셀에 이미지 저장하기 
wb = openpyxl.load_workbook(fx_name)
ws = wb.worksheets[0]

for a in range(2, no+1) : 
    ws.row_dimensions[a].height =  250
    
ws.column_dimensions['C'].width = 50


nor = 2
for a in img_list1 : 
    res = requests.get(a)
    image_data = BytesIO(res.content)
    img = openpyxl.drawing.image.Image(image_data)
    ws.add_image(img, anchor='C%s' %nor)
    nor +=1
wb.save(fx_name)
#print('엑셀에 사진 첨부 완료') 

#10 종료 타임스탬프 찍기
e_time = time.time()

#11.전체시간 계산하기
total_time = e_time - s_time 

#12.수집된 건수, 전체 소요시간 사용자에게 알려주기 
print('+'*100)
print('웹페이지에 존재하는 총 건수는 %s건 이었습니다' %len(con_list))
print('요청하신 총 %s 건 중 %s 건 수집 완료 하였습니다' %(cnt, no-1))
print('총 소요시간은 %s 초 입니다' %round(total_time, 2))
print('='*100)



----------------------------------------------------------------------------------------------------
1.판매순위: 1
2.제품소개: 청정원 리치부어스트 1.5kg
3.원래가격: 
4.판매가격: 14,900원
5.할인율: 제공된 정보 없음

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위: 2
2.제품소개: [에그머니](카드가능)(에그머니) 온라인 게임상품권 5만원
3.원래가격: 50,000원
4.판매가격: 45,000원
5.할인율: 10%

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위: 3
2.제품소개: [배스킨라빈스](배스킨라빈스) 이달의맛 포함 패밀리 아이스크림
3.원래가격: 22,000원
4.판매가격: 17,600원
5.할인율: 20%

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
1.판매순위: 4
2.제품소개: 영암 꿀달

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
웹페이지에 존재하는 총 건수는 200건 이었습니다
요청하신 총 200 건 중 200 건 수집 완료 하였습니다
총 소요시간은 175.26 초 입니다
